In [10]:
import re

import numpy as np
import pandas as pd
from underthesea import word_tokenize

# Lab
---

```
0. Code hai thuật toán MED (page slide 14, 24)

1. Tạo corpus Poetry = (Sóng - Xuân Quỳnh) --> Bóc tách từ vựng
   - Từ đơn
   - Từ ghép (2 từ đều có trong câu)
Lọc các từ có trong từ điển (ref pyenchant sẵn + gg microsoft + ...)

2. Test nối câu đầu vào: s = "Khi bão đổ bộ, Xóng gió nổi lên"
    - Noisy chanel (MED ~ D - L)
    - N-Grams (Bi/Tri)
```
---

# 0. Hai thuật toán MED

In [11]:
s1 = 'intention'
s2 = 'execution'

## Min Edit Distance (Levenshtein)

In [12]:
def levenshtein_distance(s1, s2):
    n = len(s1)
    m = len(s2)
    
    # Tạo ma trận dp bằng list comprehension
    dp = [[0 for _ in range(m + 1)] for _ in range(n + 1)]
    
    # Khởi tạo hàng đầu tiên (insert)
    for j in range(m + 1):
        dp[0][j] = j
    
    # Khởi tạo cột đầu tiên (delete)
    for i in range(n + 1):
        dp[i][0] = i
    
    # Điền ma trận dp
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            if s1[i-1] == s2[j-1]:
                cost = 0
            else:
                cost = 2
            
            dp[i][j] = min(
                dp[i-1][j] + 1,      # Delete (xóa ký tự từ s1)
                dp[i][j-1] + 1,      # Insert (chèn ký tự vào s1)
                dp[i-1][j-1] + cost  # Substitute (thay thế hoặc match)
            )

    min_operations = dp[n][m]
    
    return min_operations

In [13]:
levenshtein_distance(s1, s2)

8

## Adding Backtrace to Minimum Edit Distance

In [14]:
def levenshtein_with_backtrace(s1, s2):
    n = len(s1)
    m = len(s2)
    
    # Ma trận dp - thay np.zeros bằng list comprehension
    dp = [[0 for _ in range(m + 1)] for _ in range(n + 1)]
    
    # Khởi tạo ma trận backpointer (ptr)
    # 0: DIAG, 1: LEFT (insert), 2: DOWN (delete)
    ptr = [[0 for _ in range(m + 1)] for _ in range(n + 1)]
    
    for j in range(m + 1):
        dp[0][j] = j
        if j > 0:
            ptr[0][j] = 1  # LEFT (insert)
    
    for i in range(n + 1):
        dp[i][0] = i
        if i > 0:
            ptr[i][0] = 2  # DOWN (delete)
    
    # Fill DP table với backpointer
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            if s1[i-1] == s2[j-1]:
                substitute_cost = 0
            else:
                substitute_cost = 2
            
            # Tính cost cho 3 operations
            delete_cost = dp[i-1][j] + 1   # DOWN (delete)
            insert_cost = dp[i][j-1] + 1   # LEFT (insert)
            subst_cost = dp[i-1][j-1] + substitute_cost   # DIAG (substitute/match)
            
            # Chọn operation có cost nhỏ nhất
            min_cost = min(delete_cost, insert_cost, subst_cost)
            dp[i][j] = min_cost

            # Lưu backpointer
            if min_cost == subst_cost:
                ptr[i][j] = 0  # DIAG
            elif min_cost == insert_cost:
                ptr[i][j] = 1  # LEFT
            else:
                ptr[i][j] = 2  # DOWN
    
    # Backtrace để tìm alignment
    alignment_ops = []
    i, j = n, m
    
    while i > 0 or j > 0:
        if i == 0:
            alignment_ops.append(('INSERT', s2[j-1], j-1))
            j -= 1
        elif j == 0:
            alignment_ops.append(('DELETE', s1[i-1], i-1))
            i -= 1
        else:
            direction = ptr[i][j]

            if direction == 0:  # DIAG
                if s1[i-1] == s2[j-1]:
                    alignment_ops.append(('MATCH', s1[i-1], i-1, j-1))
                else:
                    alignment_ops.append(('SUBSTITUTE', s1[i-1], '->', s2[j-1], i-1, j-1))
                i -= 1
                j -= 1
            elif direction == 1:  # LEFT (insert)
                alignment_ops.append(('INSERT', s2[j-1], j-1))
                j -= 1
            else:  # DOWN (delete)
                alignment_ops.append(('DELETE', s1[i-1], i-1))
                i -= 1
    
    alignment_ops.reverse()

    return dp[n][m], alignment_ops

In [15]:
min_ops, alignment = levenshtein_with_backtrace(s1, s2)

print(min_ops)

8


# 1. 
Tạo corpus Poetry = (Sóng - Xuân Quỳnh) --> Bóc tách từ vựng
   - Từ đơn
   - Từ ghép (2 từ đều có trong câu)

Lọc các từ có trong từ điển (ref pyenchant sẵn + gg microsoft + ...)

### Manually

In [16]:
corpus = """
Dữ dội và dịu êm
Ồn ào và lặng lẽ
Sông không hiểu nổi mình
Sóng tìm ra tận bể

Ôi con sóng ngày xưa
Và ngày sau vẫn thế
Nỗi khát vọng tình yêu
Bồi hồi trong ngực trẻ

Trước muôn trùng sóng bể
Em nghĩ về anh, em
Em nghĩ về biển lớn
Từ nơi nào sóng lên?

Sóng bắt đầu từ gió
Gió bắt đầu từ đâu?
Em cũng không biết nữa
Khi nào ta yêu nhau

Con sóng dưới lòng sâu
Con sóng trên mặt nước
Ôi con sóng nhớ bờ
Ngày đêm không ngủ được
Lòng em nhớ đến anh
Cả trong mơ còn thức

Dẫu xuôi về phương bắc
Dẫu ngược về phương nam
Nơi nào em cũng nghĩ
Hướng về anh - một phương

Ở ngoài kia đại dương
Trăm nghìn con sóng đó
Con nào chẳng tới bờ
Dù muôn vời cách trở

Cuộc đời tuy dài thế
Năm tháng vẫn đi qua
Như biển kia dẫu rộng
Mây vẫn bay về xa

Làm sao được tan ra
Thành trăm con sóng nhỏ
Giữa biển lớn tình yêu
Để ngàn năm còn vỗ
"""


In [17]:
clean_text = re.sub(r'[^A-Za-zÀ-ỹ0-9\s]', '', corpus)
clean_text = re.sub(r'\s+', ' ', clean_text).strip()

clean_text

'Dữ dội và dịu êm Ồn ào và lặng lẽ Sông không hiểu nổi mình Sóng tìm ra tận bể Ôi con sóng ngày xưa Và ngày sau vẫn thế Nỗi khát vọng tình yêu Bồi hồi trong ngực trẻ Trước muôn trùng sóng bể Em nghĩ về anh em Em nghĩ về biển lớn Từ nơi nào sóng lên Sóng bắt đầu từ gió Gió bắt đầu từ đâu Em cũng không biết nữa Khi nào ta yêu nhau Con sóng dưới lòng sâu Con sóng trên mặt nước Ôi con sóng nhớ bờ Ngày đêm không ngủ được Lòng em nhớ đến anh Cả trong mơ còn thức Dẫu xuôi về phương bắc Dẫu ngược về phương nam Nơi nào em cũng nghĩ Hướng về anh một phương Ở ngoài kia đại dương Trăm nghìn con sóng đó Con nào chẳng tới bờ Dù muôn vời cách trở Cuộc đời tuy dài thế Năm tháng vẫn đi qua Như biển kia dẫu rộng Mây vẫn bay về xa Làm sao được tan ra Thành trăm con sóng nhỏ Giữa biển lớn tình yêu Để ngàn năm còn vỗ'

In [18]:
with open("assets/Viet74K.txt", "r", encoding="utf-8") as f:
    dictionary = f.read().splitlines()
    dictionary = [w.lower() for w in dictionary]
    
print(f"Số lượng từ trong từ điển: {len(dictionary)}")

Số lượng từ trong từ điển: 73902


In [19]:
# Tách từ đơn
single_word_list = clean_text.split()

# Tách từ ghép
double_word_list = [" ".join([single_word_list[i], single_word_list[i+1]]) for i in range(len(single_word_list)-1)]

# Lọc từ có trong từ điển
manually_word_list = [word for word in single_word_list if word.lower() in dictionary] + [word for word in double_word_list if word.lower() in dictionary]

for w in manually_word_list:
    print(w)

Dữ
dội
và
dịu
êm
Ồn
ào
và
lặng
lẽ
Sông
không
hiểu
nổi
mình
Sóng
tìm
ra
tận
bể
Ôi
con
sóng
ngày
xưa
Và
ngày
sau
vẫn
thế
Nỗi
khát
vọng
tình
yêu
Bồi
hồi
trong
ngực
trẻ
Trước
muôn
trùng
sóng
bể
Em
nghĩ
về
anh
em
Em
nghĩ
về
biển
lớn
Từ
nơi
nào
sóng
lên
Sóng
bắt
đầu
từ
gió
Gió
bắt
đầu
từ
đâu
Em
cũng
không
biết
nữa
Khi
nào
ta
yêu
nhau
Con
sóng
dưới
lòng
sâu
Con
sóng
trên
mặt
nước
Ôi
con
sóng
nhớ
bờ
Ngày
đêm
không
ngủ
được
Lòng
em
nhớ
đến
anh
Cả
trong
mơ
còn
thức
Dẫu
xuôi
về
phương
bắc
Dẫu
ngược
về
phương
nam
Nơi
nào
em
cũng
nghĩ
Hướng
về
anh
một
phương
Ở
ngoài
kia
đại
dương
Trăm
nghìn
con
sóng
đó
Con
nào
chẳng
tới
bờ
Dù
muôn
vời
cách
trở
Cuộc
đời
tuy
dài
thế
Năm
tháng
vẫn
đi
qua
Như
biển
kia
dẫu
rộng
Mây
vẫn
bay
về
xa
Làm
sao
được
tan
ra
Thành
trăm
con
sóng
nhỏ
Giữa
biển
lớn
tình
yêu
Để
ngàn
năm
còn
vỗ
Dữ dội
Ồn ào
lặng lẽ
tìm ra
ngày xưa
ngày sau
khát vọng
tình yêu
Bồi hồi
muôn trùng
anh em
em Em
bắt đầu
đầu từ
bắt đầu
đầu từ
nữa Khi
Khi nào
mặt nước
Ngày đêm
được Lòng
anh Cả
đại dương
Trăm 

## Lib

In [20]:
word_list_lib = word_tokenize(clean_text)
for w in word_list_lib:
    print(w)

Dữ dội
và
dịu êm
Ồn ào
và
lặng lẽ
Sông
không
hiểu
nổi
mình
Sóng
tìm
ra
tận
bể
Ôi
con
sóng
ngày xưa
Và
ngày sau
vẫn
thế Nỗi
khát vọng
tình yêu
Bồi hồi
trong
ngực
trẻ
Trước
muôn trùng
sóng
bể Em
nghĩ
về
anh em
Em
nghĩ
về
biển
lớn
Từ
nơi
nào
sóng
lên
Sóng
bắt đầu
từ
gió Gió
bắt đầu
từ
đâu Em
cũng
không
biết
nữa
Khi
nào
ta
yêu
nhau
Con
sóng
dưới
lòng
sâu
Con
sóng
trên
mặt nước
Ôi
con
sóng
nhớ
bờ
Ngày đêm
không
ngủ
được
Lòng
em
nhớ
đến
anh Cả
trong
mơ
còn
thức
Dẫu xuôi
về
phương
bắc
Dẫu
ngược
về
phương
nam Nơi
nào
em
cũng
nghĩ
Hướng
về
anh
một
phương
Ở
ngoài
kia
đại dương
Trăm nghìn
con
sóng
đó
Con
nào
chẳng
tới
bờ
Dù
muôn vời
cách trở
Cuộc đời
tuy
dài thế
Năm tháng
vẫn
đi
qua
Như
biển
kia
dẫu
rộng
Mây
vẫn
bay
về
xa
Làm sao
được
tan
ra
Thành trăm
con
sóng
nhỏ
Giữa
biển
lớn
tình yêu
Để
ngàn
năm
còn
vỗ


# 2.
Test nối câu đầu vào: s = "Khi bão đổ bộ, Xóng gió nổi lên"

    - Noisy chanel (MED ~ D - L)
    - N-Grams (Bi/Tri)